# Creación y agrupación de los datos para el dataset

A pesar de que la limpieza general del dataset se llevará a cabo en la solución principal creada en RStudio, queremos complementar la información extraída en la Práctica 1. Esta sería el datatset inicial "cost_of_living_countries.csv".

Para ello, añadiremos a los datos de [Expatistan](https://www.expatistan.com/check/humanity) los siguientes datos recopilados mediante Excel:

- El PIB anual por país de la web [Expansion/PIB](https://datosmacro.expansion.com/pib)

- El Salario Mínimo Interprofesional por país de la web [Expansion/SMI](https://datosmacro.expansion.com/smi)

Además, para los datos inconclusos del SMI usaremos en la solución de RStudio, en el apartado de limpeiza los siguientes datos:

- La Cotización de divisas a día 10/06/2023 frente al Euro en [Expansion/divisas](https://datosmacro.expansion.com/divisas)

Previamente a este notebook, se han hecho las siguientes modificaciones a cada subdataset mediante el Power Query de Excel para agilizar el proceso:

- Eliminar columnas innecesarias y/o vacías

- Renombrar columnas por nombres más explicativos (ex: SMI y SMI2 por SMI (dolares) y SMI (euros))

## 0. Imports y settings

In [1]:
pip install mtranslate

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from mtranslate import translate

## 1. Carga de los datos
Cargaremos el Excel que recopila, en 3 hojas distintas, los valores el PIB, el SMI y la cotización de divisas por país y el dataset original de la práctica 1. Para la primera parte, solo cargaremos el PIB y el SMI:

In [18]:
# Primeros cargamos las hojas del excel de ampliación
PIB = pd.read_excel("../datasets/PBI_SMI_divisas.xlsx", sheet_name = "PIB", index_col=None)
SMI = pd.read_excel("../datasets/PBI_SMI_divisas.xlsx", sheet_name = "SMI", index_col=None)
SMI = SMI.drop(['SMI Mon. Local', 'SMI Mon. Local2'], axis=1)

In [19]:
# Ahora cargaremos el dataset original
cost_countries = pd.read_csv("../datasets/cost_of_living_countries.csv", index_col=None)
cost_countries['Country'] = cost_countries['Country'].str.upper()

## 2. Unión de los datasets de ampliación
Dado que todos presentan la misma columna países, procederemos a unirlos en un único dataset:

In [20]:
dataset_ampliacion = pd.merge(PIB, SMI, on='Países', how='left')

In [21]:
dataset_ampliacion.head()

,Países,PIB anual,SMI (dolares),SMI (euros)
0,España [+],1.328.922 M€,"1.343,9 $","1.260,0 €"
1,Alemania [+],3.869.900 M€,"2.119,3 $","1.987,0 €"
2,Reino Unido [+],2.913.623 M€,"1.952,7 $","1.929,2 €"
3,Francia [+],2.639.092 M€,"1.823,1 $","1.709,3 €"
4,Italia [+],1.909.154 M€,NaN,NaN


## 3. Ajustar la variables común Países a la variable Country
El puente de unión entre el dataset original y el de ampliación son los países. Sin embargo, están en diferentes idiomas: el original en inglés y el de ampliación en español y con caracteres extraños. Procederemos a limpiar estos caracteres y traducir los nombres de los países al inglés:

In [22]:
# Eliminamos los caracteres " [+]" del final de cada nombre de país y los países repetidos
dataset_ampliacion['Países'] = dataset_ampliacion['Países'].str[:-4]
dataset_ampliacion = dataset_ampliacion.drop_duplicates(subset='Países')

In [23]:
# Definimos el traductor
def translate_country(nombre_pais):
    try:
        nombre_ingles = translate(nombre_pais, 'en', 'auto')
        return nombre_ingles.upper()
    except Exception as e:
        print(f"Error al traducir {nombre_pais}: {str(e)}")

# Amplicamos el traductor
dataset_ampliacion['Country'] = dataset_ampliacion['Países'].apply(translate_country)

In [24]:
# Dado que el traductor tiene sus pequeños fallos, los corregiremos
dataset_ampliacion['Country'] = dataset_ampliacion['Country'].replace({' Y ':' AND '}, regex=True)
dataset_ampliacion['Country'] = dataset_ampliacion['Country'].replace({'SWISS':'SWITZERLAND'}, regex=True)
dataset_ampliacion = dataset_ampliacion.drop('Países', axis=1)

In [25]:
dataset_ampliacion.head()

,PIB anual,SMI (dolares),SMI (euros),Country
0,1.328.922 M€,"1.343,9 $","1.260,0 €",SPAIN
1,3.869.900 M€,"2.119,3 $","1.987,0 €",GERMANY
2,2.913.623 M€,"1.952,7 $","1.929,2 €",UNITED KINGDOM
3,2.639.092 M€,"1.823,1 $","1.709,3 €",FRANCE
4,1.909.154 M€,NaN,NaN,ITALIA


## 4. Unión final

In [26]:
final_dataset = pd.merge(cost_countries, dataset_ampliacion, on='Country', how='left')

In [27]:
paises_nan_smi = final_dataset[final_dataset['SMI (euros)'].isna()][['Country', 'PIB anual', 'SMI (euros)']]

In [28]:
# Paises con SMI nulo pero PBI no nulo
set(paises_nan_smi[paises_nan_smi['PIB anual'].notna()]['Country'])

{'AUSTRIA',
 'DENMARK',
 'FINLAND',
 'GUYANA',
 'NORWAY',
 'SINGAPORE',
 'SWEDEN',
 'UNITED ARAB EMIRATES',
 'VENEZUELA',
 'VIETNAM'}

In [29]:
# Paises con el SMI nulo
set(paises_nan_smi['Country'])

{'AUSTRIA',
 'BERMUDA',
 'CAYMAN ISLANDS',
 'DENMARK',
 'FIJI',
 'FINLAND',
 'GUYANA',
 'ITALY',
 'KOSOVO',
 'MACAO',
 'MAURITIUS',
 'NORWAY',
 'PALESTINIAN TERRITORY',
 'PHILIPPINES',
 'SINGAPORE',
 'SWEDEN',
 'UNITED ARAB EMIRATES',
 'UNITED STATES',
 'VENEZUELA',
 'VIETNAM'}

In [30]:
final_dataset.to_csv('cost_of_living_countries_updated.csv', index=False)